# Image Labeling & Model Finetuning

## 1. Set up 

In [1]:
#%%capture
#!pip install matplotlib

In [2]:
#%%capture
#!pip install seaborn

In [3]:
#%%capture
#!pip install torch

In [4]:
#%%capture
#!pip install ultralytics

In [5]:
#%%capture
#!pip install tqdm

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
import os
import shutil
import csv
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import random
import torch
from ultralytics.data import utils as data_utils
from pathlib import Path
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from tqdm import tqdm

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
0


In [4]:
%matplotlib inline

## 2. Define Paths to Folders & Files

In [5]:
# Get current working directory
current_dir = os.getcwd()

# Go up one level to the parent directory
parent_dir = os.path.dirname(current_dir)

# Set path for the source folder (you can adjust as needed)
source_folder = os.path.join(parent_dir, "idx_images")
labeling_folder = os.path.join(parent_dir, "img_labeling")

csv_out_path = os.path.join(source_folder, "image_metadata_weather.csv")
csv_train = os.path.join(labeling_folder, "train_for_training.csv")
csv_test = os.path.join(labeling_folder, "test_for_training.csv")
csv_val = os.path.join(labeling_folder, "val_for_training.csv")

print(f"Source images located in: {source_folder}")


Source images located in: C:\Users\A\Documents\XX_GitHub_Repo\data-waves\idx_images


In [6]:
image_files = sorted([f for f in os.listdir(source_folder) if f.endswith('.png')])

In [7]:
print(len(image_files), "images found")

27592 images found


In [11]:
# YOLO model (load once)
yolo_model = YOLO("yolov8n.pt")

## 3. Finetuning YOLO Model using stratified Train, Test, Val set and created labels

In [12]:
# New dataset root where YOLO dataset will be automatically created
dataset_root = os.path.join(parent_dir, "waves_yolo_dataset_1000_small")
os.makedirs(dataset_root, exist_ok=True)



In [13]:
# Load lists of image names
# -------------------------------------------------------------
def load_image_list(csv_path):
    df = pd.read_csv(csv_path)
    if "new_image_name" in df.columns:
        col = "new_image_name"
    elif "image_name" in df.columns:
        col = "image_name"
    else:
        raise ValueError(f"No image name column in {csv_path}")
    return df[col].dropna().astype(str).tolist()

# Load ALL images from CSVs
train_imgs_full = load_image_list(csv_train)
val_imgs_full   = load_image_list(csv_val)
test_imgs_full  = load_image_list(csv_test)

In [14]:
# OPTIONAL: Subsample: 500 train, 200 val, 200 test
# (shuffled so you don't just get the first rows)
# -------------------------------------------------------------
random.seed(42)  # for reproducibility

train_imgs = random.sample(train_imgs_full, min(1000, len(train_imgs_full)))
val_imgs   = random.sample(val_imgs_full,  min(200, len(val_imgs_full)))
test_imgs  = random.sample(test_imgs_full, min(200, len(test_imgs_full)))

print(f"Using subset: train={len(train_imgs)}, val={len(val_imgs)}, test={len(test_imgs)}")


Using subset: train=1000, val=200, test=200


In [15]:
# Write YOLO split txt files with ABSOLUTE PATHS
# -------------------------------------------------------------
def write_split_file(img_list, out_path):
    with open(out_path, "w") as f:
        for name in img_list:
            full_path = os.path.abspath(os.path.join(source_folder, name))
            f.write(full_path + "\n")

train_txt = os.path.join(dataset_root, "train.txt")
val_txt   = os.path.join(dataset_root, "val.txt")
test_txt  = os.path.join(dataset_root, "test.txt")

write_split_file(train_imgs, train_txt)
write_split_file(val_imgs, val_txt)
write_split_file(test_imgs, test_txt)

print("Wrote split files:")
print(train_txt)
print(val_txt)
print(test_txt)


Wrote split files:
C:\Users\A\Documents\XX_GitHub_Repo\data-waves\waves_yolo_dataset_1000_small\train.txt
C:\Users\A\Documents\XX_GitHub_Repo\data-waves\waves_yolo_dataset_1000_small\val.txt
C:\Users\A\Documents\XX_GitHub_Repo\data-waves\waves_yolo_dataset_1000_small\test.txt


In [16]:
# Create YOLO data.yaml
# IMPORTANT: 'path' tells YOLO where labels live.
# -------------------------------------------------------------
# Normalize Windows paths before inserting into f-string
train_txt_norm = train_txt.replace("\\", "/")
val_txt_norm   = val_txt.replace("\\", "/")
test_txt_norm  = test_txt.replace("\\", "/")
parent_dir_norm = parent_dir.replace("\\", "/")

# Create YOLO data.yaml
data_yaml_path = os.path.join(dataset_root, "03_waves.yaml")

yaml_text = f"""
# Dataset config without copying images

train: {train_txt_norm}
val: {val_txt_norm}
test: {test_txt_norm}

names:
  0: wave
nc: 1
"""

with open(data_yaml_path, "w") as f:
    f.write(yaml_text)

print("Created data.yaml at:", data_yaml_path)


Created data.yaml at: C:\Users\A\Documents\XX_GitHub_Repo\data-waves\waves_yolo_dataset_1000_small\03_waves.yaml


##### Load model and checking if model finds labels

In [17]:
# Load pretrained model first try nano than small or medium
base_model = YOLO("yolov8s.pt")

info = data_utils.check_det_dataset(data_yaml_path)
print("Train source:", info["train"])
print("Val source:  ", info["val"])
print("Test source: ", info["test"])

Train source: C:\Users\A\Documents\XX_GitHub_Repo\data-waves\waves_yolo_dataset_1000_small\train.txt
Val source:   C:\Users\A\Documents\XX_GitHub_Repo\data-waves\waves_yolo_dataset_1000_small\val.txt
Test source:  C:\Users\A\Documents\XX_GitHub_Repo\data-waves\waves_yolo_dataset_1000_small\test.txt


In [18]:
def count_missing_labels(txt_file, label_folder):
    missing = []
    with open(txt_file) as f:
        for line in f:
            img_path = line.strip()
            img_name = Path(img_path).name
            label_name = img_name.replace(".png", ".txt")
            label_path = Path(label_folder) / label_name
            if not label_path.exists():
                missing.append(label_name)
    return missing

labels_folder = r"C:/Users/A/Documents/XX_GitHub_Repo/data-waves/labels"

missing_train = count_missing_labels(info["train"], labels_folder)
missing_val   = count_missing_labels(info["val"], labels_folder)
missing_test  = count_missing_labels(info["test"], labels_folder)

print("Missing train labels:", len(missing_train))
print("Missing val labels:  ", len(missing_val))
print("Missing test labels: ", len(missing_test))

if missing_train:
    print("Example missing label:", missing_train[:5])

Missing train labels: 1000
Missing val labels:   200
Missing test labels:  200
Example missing label: ['img_14237.txt', 'img_15383.txt', 'img_04648.txt', 'img_16453.txt', 'img_06242.txt']


##### Run finetuning

In [19]:
# Device selection (GPU if available, else CPU)
# ------------------------------------------------------------------
if torch.cuda.is_available():
    device = 0  # GPU index for ultralytics (0 = first GPU)
    print("✅ CUDA is available. Training on GPU.")
else:
    device = "cpu"
    print("⚠️ CUDA not available. Training on CPU (slower).")

✅ CUDA is available. Training on GPU.


In [20]:

# Give a name to the run
run_name = "03_waves_yolov8s-1000_150e"

# Fine-tune the model
base_model.train(
    data=data_yaml_path,
    device = device,
    epochs=150,
    imgsz=(512, 2048), #(1024,5069) - time limit exceeded, not enough comutational performance on cpu.
    batch=1,
    name=run_name,
    patience=0,
    save=True
    #amp=False
)

New https://pypi.org/project/ultralytics/8.3.231 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.230  Python-3.9.21 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\haas\data-waves\waves_yolo_dataset_1000_small\03_waves.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=(512, 2048), int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scal

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022ACA0D9610>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [21]:
print("Contents of runs/detect:", os.listdir("runs/detect"))

Contents of runs/detect: ['.ipynb_checkpoints', '01_waves_yolov8n-500', '02_waves_yolov8n-1000', '02_waves_yolov8n-1000_150e', '03_waves_yolov8s-1000_150e', 'eval_base', 'eval_base_01_waves_yolov8n', 'eval_finetuned_01', 'eval_finetuned_01_waves_yolov8n', 'wave1_yolov8n', 'wave1_yolov8n2', 'wave1_yolov8n3', 'wave1_yolov8n4', 'wave1_yolov8n5', 'wave1_yolov8n6', 'wave1_yolov8n7']


In [20]:
# rerun the missing validation step only to get metrics

model_03_waves_yolov8s_1000_150e = YOLO("03_waves_yolov8s-1000_150e.pt")  #last weights from 05_waves_yolov8s-5000_150e to run the validation process
results = model_03_waves_yolov8s_1000_150e.val(data=data_yaml_path,device = device, imgsz=2048, split="val")
print(results.results_dict)

Ultralytics 8.3.168  Python-3.9.21 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce MX450, 2048MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 2247.6176.0 MB/s, size: 2362.1 KB)


val: Scanning C:\Users\A\Documents\XX_GitHub_Repo\data-waves\idx_images.cache... 200 images, 10 backgrounds, 0 corrupt:
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [02:43


                   all        200     127491     0.0311     0.0126     0.0207    0.00619
Speed: 5.1ms preprocess, 799.9ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to runs\detect\val52
{'metrics/precision(B)': np.float64(0.031071691961339905), 'metrics/recall(B)': np.float64(0.01255774917445153), 'metrics/mAP50(B)': np.float64(0.020732190950238875), 'metrics/mAP50-95(B)': np.float64(0.006190454152466257), 'fitness': np.float64(0.007644627832243519)}
